In [1]:
#!pip install ccxt pandas numpy plotly python-dotenv nest-asyncio kaleido

In [ ]:
import asyncio
import ccxt.async_support as ccxt
import pandas as pd
import numpy as np
from datetime import datetime 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from dotenv import load_dotenv
import nest_asyncio
from tqdm import tqdm

# Jupyter için asyncio desteği
nest_asyncio.apply()

class TradingScanner:
    def __init__(self):
        load_dotenv()
        self.api_key = os.getenv('BINANCE_API_KEY')
        self.api_secret = os.getenv('BINANCE_API_SECRET')
        self.exchange = ccxt.binance({
        'apiKey': self.api_key,
        'secret': self.api_secret,
        'enableRateLimit': True,
        'timeout': 30000,  # Timeout süresini 30 saniyeye çıkaralım
        'options': {
            'defaultType': 'spot',
            'adjustForTimeDifference': True,
            'recvWindow': 60000  # Receive window süresini artıralım
        }
    })
        
        self.timeframes = ['15m', '1h', '4h', '1d']
        self.min_volume = 5_000_000
        
        self.timeframe_names = {
            '15m': '15 Dakika',
            '1h': '1 Saat',
            '4h': '4 Saat',
            '1d': '1 Gün'
        }
        
        self.timeframe_limits = {
            '15m': 300,    # 1000'den 300'e düşürüldü
            '1h': 200,     # 500'den 200'e düşürüldü
            '4h': 200,     # Aynı
            '1d': 200      # Aynı
        }
        
        self.lookback_periods = {
            '15m': 96,     # 192'den 96'ya düşürüldü (24 saat)
            '1h': 72,      # 120'den 72'ye düşürüldü (3 gün)
            '4h': 60,      # 90'dan 60'a düşürüldü (10 gün)
            '1d': 60       # 89'dan 60'a düşürüldü (2 ay)
        }
        
        self.excluded_coins = {
            'USDC', 'USDT', 'USTC', 'FDUSD', 'BUSD', 
            'TUSD', 'DAI', 'UST', 'USDD', 'USDP', 'SUSD'
        }
        
    async def get_active_markets(self):
        print("\nAktif marketler taranıyor...")
        markets = await self.exchange.load_markets()
        active_pairs = []
        
        usdt_pairs = [s for s in markets if s.endswith('/USDT')]
        for symbol in tqdm(usdt_pairs, desc="Market Taraması", ncols=100):
            coin = symbol.split('/')[0]
            if coin not in self.excluded_coins:
                try:
                    ticker = await self.exchange.fetch_ticker(symbol)
                    if ticker['quoteVolume'] >= self.min_volume:
                        active_pairs.append(symbol)
                except Exception as e:
                    print(f"Hata ({symbol}): {str(e)}")
                    continue
                    
                await asyncio.sleep(0.2)  # 0.1'den 0.2'ye çıkarıldı
        
        print(f"\nToplam {len(active_pairs)} aktif market bulundu")
        print(f"İşlem yapılacak marketler: {', '.join(active_pairs)}\n")
                
        return active_pairs

    def calculate_squeeze(self, ohlcv, period=20):
        if len(ohlcv) < period:
            return False
            
        close = ohlcv[:, 4]
        high = ohlcv[:, 2]
        low = ohlcv[:, 3]
        
        middle = np.mean(close[-period:])
        stddev = np.std(close[-period:])
        bb_upper = middle + (2 * stddev)
        bb_lower = middle - (2 * stddev)
        
        tr = np.maximum.reduce([
            high[-period:] - low[-period:],
            np.abs(high[-period:] - np.roll(close[-period:], 1)),
            np.abs(low[-period:] - np.roll(close[-period:], 1))
        ])
        atr = np.mean(tr)
        kc_upper = middle + (1.5 * atr)
        kc_lower = middle - (1.5 * atr)
        
        return (bb_lower > kc_lower) and (bb_upper < kc_upper)

    def calculate_fibonacci_gradient(self, df, timeframe):

        lookback = self.lookback_periods[timeframe]
        gradients = np.zeros(len(df))
        
        for i in range(lookback, len(df)):
            window = df.iloc[i-lookback:i]
            high = window['high'].astype(float)
            low = window['low'].astype(float)
            close = float(df['close'].iloc[i])
            
            fib_max = high.max()
            fib_min = low.min()
            max_min_diff = fib_max - fib_min
            
            if max_min_diff <= 0:
                continue
                
            fib_sell = fib_max - (max_min_diff * 0.382)
            fib_buy = fib_max - (max_min_diff * 0.707)
            
            if close >= fib_sell:
                sell_diff = ((close - fib_sell) / max_min_diff * 100)
                gradient = ((sell_diff * 2617.801) + 100000) / 2
            elif close <= fib_buy:
                buy_diff = ((fib_buy - close) / max_min_diff * 100)
                gradient = ((buy_diff * -2617.801) + 100000) / 2
            else:
                gradient = 50000
                
            gradients[i] = (100 - gradient/1000) * -1
            
        return gradients
    
    async def analyze_market(self, symbol):
        results = {}
        ohlcv_data = {}
        timeframe_data = {}
        
        # Önce tüm timeframe'ler için veri çekelim
        for timeframe in tqdm(self.timeframes, desc=f"{symbol}", leave=False, ncols=100):
            try:
                ohlcv = await self.exchange.fetch_ohlcv(
                    symbol, 
                    timeframe, 
                    limit=self.timeframe_limits[timeframe]
                )
                
                if not ohlcv or len(ohlcv) < self.lookback_periods[timeframe]:
                    print(f"Yetersiz veri: {symbol} - {timeframe}")
                    return results, ohlcv_data  # Eğer herhangi bir timeframe'de veri yoksa coin'i atla
                    
                timeframe_data[timeframe] = {
                    'ohlcv': ohlcv,
                    'array': np.array(ohlcv),
                    'df': pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                }
                
            except Exception as e:
                print(f"Veri çekme hatası ({symbol} - {timeframe}): {str(e)}")
                return results, ohlcv_data
                
            await asyncio.sleep(0.5)

        # Tüm timeframe'ler için Fibonacci gradient kontrolü
        all_signals = {}
        
        for timeframe, data in timeframe_data.items():
            # Fibonacci gradient kontrolü
            gradients = self.calculate_fibonacci_gradient(data['df'], timeframe)
            last_gradient = gradients[-1]
            
            # Sinyal kontrolü
            if last_gradient <= -70.7:
                signal = "Alış"
            elif last_gradient >= -38.2:
                signal = "Satış"
            else:
                return results, ohlcv_data  # Herhangi bir timeframe'de sinyal yoksa coin'i atla
                
            all_signals[timeframe] = {
                'signal': signal,
                'gradient': last_gradient
            }
        
        # Tüm sinyaller aynı mı kontrol et (hepsi alış veya hepsi satış)
        first_signal = list(all_signals.values())[0]['signal']
        if not all(signal['signal'] == first_signal for signal in all_signals.values()):
            return results, ohlcv_data
            
        # Buraya gelindiyse tüm sinyaller aynı, şimdi squeeze kontrolü yapalım
        has_squeeze = False
        for timeframe, data in timeframe_data.items():
            if self.calculate_squeeze(data['array']):
                has_squeeze = True
                break
        
        # Eğer squeeze varsa tüm verileri kaydet
        if has_squeeze:
            results[symbol] = all_signals
            ohlcv_data = {tf: data['ohlcv'] for tf, data in timeframe_data.items()}
            print(f"\nSinyal bulundu - {symbol}:")
            for tf, signal in all_signals.items():
                print(f"✓ {self.timeframe_names[tf]}: {signal['signal']} ({signal['gradient']:.2f})")
        
        return results, ohlcv_data

    def plot_analysis(self, symbol, data, ohlcv_data):
        fig = make_subplots(
            rows=2, 
            cols=2, 
            subplot_titles=[f"{symbol} - {self.timeframe_names[tf]}" for tf in self.timeframes],
            vertical_spacing=0.1,
            horizontal_spacing=0.05
        )

        colors = {
            '15m': 'rgba(255, 0, 0, 0.3)', 
            '1h': 'rgba(0, 255, 0, 0.3)', 
            '4h': 'rgba(0, 0, 255, 0.3)',
            '1d': 'rgba(255, 165, 0, 0.3)'
        }

        for idx, timeframe in enumerate(self.timeframes):
            row = idx // 2 + 1
            col = idx % 2 + 1
            
            if timeframe in ohlcv_data:
                ohlcv = ohlcv_data[timeframe]
                df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                
                # Gradient hesaplama
                gradients = self.calculate_fibonacci_gradient(df, timeframe)
                df['gradient'] = gradients
                
                # Ana mum grafiği
                fig.add_trace(
                    go.Candlestick(
                        x=df['timestamp'],
                        open=df['open'],
                        high=df['high'],
                        low=df['low'],
                        close=df['close'],
                        name=timeframe,
                        showlegend=False,
                        customdata=df['gradient'],
                        hovertext=[
                            f'Tarih: {date}<br>' +
                            f'Açılış: {open:.8f}<br>' +
                            f'Yüksek: {high:.8f}<br>' +
                            f'Düşük: {low:.8f}<br>' +
                            f'Kapanış: {close:.8f}<br>' +
                            f'Gradient: {gradient:.2f}'
                            for date, open, high, low, close, gradient in zip(
                                df['timestamp'], df['open'], df['high'], 
                                df['low'], df['close'], df['gradient']
                            )
                        ],
                        hoverinfo='text'
                    ),
                    row=row, col=col
                )

                # Squeeze bölgeleri için
                for i in range(1, len(df)):
                    is_squeeze = self.calculate_squeeze(np.array(ohlcv[max(0, i-20):i+1]))
                    if is_squeeze:
                        # Her squeeze bölgesi için ayrı bir shape ekle
                        fig.add_shape(
                            type="rect",
                            x0=df['timestamp'].iloc[i-1],
                            x1=df['timestamp'].iloc[i],
                            y0=df['low'].min(),
                            y1=df['high'].max(),
                            fillcolor=colors[timeframe],
                            opacity=0.3,
                            layer="below",
                            line_width=0,
                            row=row,
                            col=col
                        )

                # Sinyal kontrolü ve gösterimi
                if timeframe in data.get(symbol, {}):
                    signal_info = data[symbol][timeframe]
                    if signal_info['signal'] == "Alış":
                        fig.add_trace(
                            go.Scatter(
                                x=[df['timestamp'].iloc[-1]],
                                y=[df['low'].iloc[-1] * 0.999],
                                mode='markers',
                                marker=dict(symbol='triangle-up', size=12, color='lime'),
                                name='Alım Sinyali',
                                showlegend=idx == 0
                            ),
                            row=row, col=col
                        )
                    elif signal_info['signal'] == "Satış":
                        fig.add_trace(
                            go.Scatter(
                                x=[df['timestamp'].iloc[-1]],
                                y=[df['high'].iloc[-1] * 1.001],
                                mode='markers',
                                marker=dict(symbol='triangle-down', size=12, color='red'),
                                name='Satım Sinyali',
                                showlegend=idx == 0
                            ),
                            row=row, col=col
                        )

                # Eksen ayarları
                fig.update_xaxes(
                    title_text="Tarih",
                    gridcolor='rgba(128,128,128,0.2)',
                    rangeslider=dict(visible=False),
                    row=row, 
                    col=col,
                    tickfont=dict(size=10)
                )
                
                fig.update_yaxes(
                    title_text="Fiyat",
                    gridcolor='rgba(128,128,128,0.2)',
                    row=row, 
                    col=col,
                    tickformat='.8f',
                    tickfont=dict(size=10)
                )

        # Genel grafik stili
        fig.update_layout(
            height=1200,
            width=1800,
            template="plotly_dark",
            paper_bgcolor='black',
            plot_bgcolor='black',
            title=dict(
                text=f"{symbol}",
                font=dict(size=28, color='white'),
                x=0.5,
                y=0.95
            ),
            showlegend=True,
            legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="right",
                x=0.99,
                bgcolor='rgba(0,0,0,0.5)',
                font=dict(size=12, color='white')
            ),
            margin=dict(t=80, b=50, l=50, r=50)
        )

        # Subplot başlıkları için renk ayarı
        for annotation in fig['layout']['annotations']:
            annotation['font'] = dict(size=14, color='white')

        return fig

    async def run_analysis(self):
        try:
            markets = await self.get_active_markets()
            
            all_results = {}
            for symbol in tqdm(markets, desc="Analiz İlerlemesi", ncols=100):
                results, ohlcv_data = await self.analyze_market(symbol)
                
                if results and symbol in results:  # results boş olmadığından emin ol
                    all_results[symbol] = results[symbol]
                    print(f"\n{symbol} Sinyal Özeti:")
                    for tf, result in results[symbol].items():
                        print(f"✓ {self.timeframe_names[tf]}: {result['signal']} ({result['gradient']:.2f})")
                    
                    fig = self.plot_analysis(symbol, results[symbol], ohlcv_data)
                    filename = f"{symbol.replace('/', '_')}_{datetime.now().strftime('%Y%m%d_%H%M')}.png"
                    fig.write_image(filename, engine="kaleido", scale=2)
            
            print("\n=== ÖZET ===")
            print(f"Toplam {len(all_results)} coin için sinyal bulundu:")
            for symbol in all_results:
                timeframe = list(all_results[symbol].keys())[0]  # Sadece bir timeframe olacak
                signal_info = all_results[symbol][timeframe]
                print(f"• {symbol} ({self.timeframe_names[timeframe]}): {signal_info['signal']} ({signal_info['gradient']:.2f})")
                    
        finally:
            await self.exchange.close()
# Jupyter için çalıştırma fonksiyonu
def run_scanner():
    scanner = TradingScanner()
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(scanner.run_analysis())
    except Exception as e:
        print(f"\nBir hata oluştu: {e}")

# Programı çalıştır
if __name__ == "__main__":
    run_scanner()


Aktif marketler taranıyor...


Market Taraması: 100%|████████████████████████████████████████████| 524/524 [03:52<00:00,  2.26it/s]



Toplam 68 aktif market bulundu
İşlem yapılacak marketler: BTC/USDT, ETH/USDT, BNB/USDT, LTC/USDT, ADA/USDT, XRP/USDT, TRX/USDT, LINK/USDT, FET/USDT, FTM/USDT, DOGE/USDT, BCH/USDT, TROY/USDT, OGN/USDT, SOL/USDT, CRV/USDT, DOT/USDT, RUNE/USDT, UNI/USDT, AVAX/USDT, AAVE/USDT, NEAR/USDT, FIL/USDT, INJ/USDT, UNFI/USDT, SHIB/USDT, ICP/USDT, MASK/USDT, RAY/USDT, DYDX/USDT, VIDT/USDT, RARE/USDT, SANTOS/USDT, PEOPLE/USDT, IMX/USDT, GLMR/USDT, APE/USDT, OP/USDT, APT/USDT, ARB/USDT, SUI/USDT, PEPE/USDT, FLOKI/USDT, PENDLE/USDT, ARKM/USDT, WLD/USDT, SEI/USDT, TIA/USDT, ORDI/USDT, 1000SATS/USDT, BONK/USDT, JUP/USDT, PYTH/USDT, WIF/USDT, BOME/USDT, ETHFI/USDT, ENA/USDT, W/USDT, SAGA/USDT, TAO/USDT, NOT/USDT, RENDER/USDT, TON/USDT, DOGS/USDT, NEIRO/USDT, TURBO/USDT, 1MBABYDOGE/USDT, EIGEN/USDT



Analiz İlerlemesi:  96%|██████████████████████████████████████████  | 65/68 [03:20<00:08,  2.89s/it]

Yetersiz veri: NEIRO/USDT - 1d


...

Yetersiz veri: TURBO/USDT - 1d


Analiz İlerlemesi:  99%|███████████████████████████████████████████▎| 67/68 [03:25<00:02,  2.70s/it]

Yetersiz veri: 1MBABYDOGE/USDT - 1d


Analiz İlerlemesi: 100%|████████████████████████████████████████████| 68/68 [03:27<00:00,  3.06s/it]

Yetersiz veri: EIGEN/USDT - 1d

=== ÖZET ===
Toplam 0 coin için sinyal bulundu:
